The following tutorial is a subset of the Auto3D [documentation](https://auto3d.readthedocs.io/en/latest/index.html). Please check the documentation for detailed usages. If you have any questions, feel free to open a issue ticket at our GitHub [repo](https://github.com/isayevlab/Auto3D_pkg).

## Install dependencies

In [1]:
# minimum dependencies
! pip install rdkit
! pip install auto3d
# dependencies for thermodynamic properties
! pip install torchani
! pip install ase
# for visualization
!pip install py3Dmol

from google.colab import output
output.enable_custom_widget_manager()

## Basic usage for getting low-energy conformers

To get the low-energy conformers, an `smi` or `SDF` file that stores the molecules is needed as the input for the package. You can import Auto3D as a library in any Python script, or run Auto3D through the commalnd line interface (CLI). They are equivalent in findig the low-energy 3D conformers.

 If you just have a handful of SMILES, it’s easy to use the `smiles2mols` function. It is a handy tool for finding the low-energy conformers for a list of SMILES. Compared with the `main` function, it sacrifices efficiency for convenience. Because `smiles2mols` uses only 1 process. Both the input and output are returned as variables within Python. It’s recommended only when the number of SMILES is less than 150; Otherwise using the combination of the `options` and `main` function will be faster.



In [2]:
from rdkit import Chem
from Auto3D.auto3D import options, smiles2mols

smiles = ['CCNCC', 'O=C(C1=CC=CO1)N2CCNCC2']
args = options(k=1, use_gpu=True)
mols = smiles2mols(smiles, args)

# get the energy and atomic positions out of the mol objects
for mol in mols:
   print(mol.GetProp('_Name'))
   print('Energy: ', mol.GetProp('E_tot'))  # unit Hartree
   conf = mol.GetConformer()
   for i in range(conf.GetNumAtoms()):
      atom = mol.GetAtomWithIdx(i)
      pos = conf.GetAtomPosition(i)
      print(f'{atom.GetSymbol()} {pos.x:.3f} {pos.y:.3f} {pos.z:.3f}')

Checking input file...
	There are 2 SMILES in the input file /tmp/tmpsgcp8fu8/smiles.smi. 
	All SMILES and IDs are valid.
Suggestions for choosing isomer_engine and optimizing_engine: 
	Isomer engine options: RDKit and Omega.
	Optimizing engine options: ANI2x, ANI2xt and AIMNET.
Enumerating cis/tran isomers for unspecified double bonds...
Enumerating R/S isomers for unspecified atomic centers...
Removing enantiomers...
Enumerating conformers/rotamers, removing duplicates...


100%|██████████| 2/2 [00:00<00:00,  3.62it/s]


Preparing for parallel optimizing... (Max optimization steps: 5000)
Total 3D conformers: 59


 10%|▉         | 498/5000 [00:39<04:11, 17.87it/s]

Total 3D structures: 59  Converged: 48   Dropped(Oscillating): 0    Active: 11


 18%|█▊        | 907/5000 [01:02<04:42, 14.47it/s]

Optimization finished at step 908:   Total 3D structures: 59  Converged: 59   Dropped(Oscillating): 0    Active: 0
Begin to select structures that satisfy the requirements...


Energy unit: Hartree if implicit.
HPNMFZURTQLUMO-UHFFFAOYSA-N
Energy:  -213.93221683810808
C -2.442 0.086 0.184
C -1.183 -0.567 -0.357
N -0.029 0.293 -0.152
C 1.227 -0.295 -0.586
C 2.390 0.625 -0.266
H -2.368 0.243 1.260
H -3.316 -0.536 -0.003
H -2.596 1.052 -0.291
H -1.049 -1.558 0.099
H -1.288 -0.729 -1.430
H 0.036 0.538 0.826
H 1.166 -0.445 -1.665
H 1.399 -1.286 -0.141
H 3.330 0.208 -0.625
H 2.240 1.595 -0.733
H 2.483 0.776 0.810
SADPINFEWFPMEA-UHFFFAOYSA-N
Energy:  -610.5301325607262
O 0.796 -1.608 1.594
C 0.717 -0.662 0.824
C 1.994 -0.116 0.283
C 3.211 -0.714 0.287
C 4.132 0.232 -0.257
C 3.406 1.337 -0.539
O 2.104 1.152 -0.216
N -0.478 -0.122 0.458
C -0.713 0.832 -0.614
C -1.889 0.383 -1.469
N -3.059 0.193 -0.629
C -2.816 -0.862 0.343
C -1.663 -0.452 1.244
H 3.403 -1.703 0.658
H 5.189 0.111 -0.409
H 3.653 2.299 -0.949
H -0.934 1.814 -0.189
H 0.167 0.918 -1.234
H -1.602 -0.535 -2.002
H -2.093 1.152 -2.212
H -3.874 -0.011 -1.188
H -2.577 -1.823 -0.132
H -3.709 -1.000 0.951
H -1.400 

In [3]:
import py3Dmol

In [4]:
xyz = Chem.MolToXYZBlock(mol)
view = py3Dmol.view(width=800, height=400)
view.addModel(xyz, 'xyz')
view.setStyle({'stick': {}})
view.zoomTo()
view.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [5]:
# we can write the conformers into an SDF file
from rdkit.Chem import SDWriter

In [6]:
path = 'example.sdf'
with SDWriter(path) as f:
  for mol in mols:
    f.write(mol)

If you have a large number of molecuels, using the `options` and the `main` functions is recommended. It’s suitable for a large input, and stores the 3D structures in a file with the name `<input_file_name>_3d.sdf`, of which the path will be printed out when auto3d finishes running. Because the `main` function uses multiprocessing, it has to be called in a `if __name__ == "__main__":` block.

## Calculate thermodynamic properties with the 3D structures

In [7]:
from Auto3D.ASE.thermo import calc_thermo

In [8]:
"""
Assuming the thermodynamic properties are calculated in 298 K and standad atomosphere.
"""

out_thermo = calc_thermo(path, "AIMNET", opt_tol=0.003)
print(out_thermo)  #enthalpy, entropy and Gibbs free energy are stored in the SDF file

  0%|          | 0/2 [00:00<?, ?it/s]

HPNMFZURTQLUMO-UHFFFAOYSA-N


 50%|█████     | 1/2 [00:02<00:02,  2.31s/it]

Enthalpy components at T = 298.00 K:
E_pot              -5821.392 eV
E_ZPE                  4.045 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.039 eV
Cv_vib (0->T)          0.113 eV
(C_v -> C_p)           0.026 eV
-------------------------------
H                  -5817.132 eV
Entropy components at T = 298.00 K and P = 101325.0 Pa:
                           S               T*S
S_trans (1 bar)    0.0016829 eV/K        0.502 eV
S_rot              0.0011215 eV/K        0.334 eV
S_elec             0.0000000 eV/K        0.000 eV
S_vib              0.0006775 eV/K        0.202 eV
S (1 bar -> P)    -0.0000011 eV/K       -0.000 eV
-------------------------------------------------
S                  0.0034807 eV/K        1.037 eV
Enthalpy components at T = 298.00 K:
E_pot              -5821.392 eV
E_ZPE                  4.045 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.039 eV
Cv_vib (0->T)          0.113 eV
(C_v -> C_p)           0.026 eV
-------------------------

100%|██████████| 2/2 [00:24<00:00, 12.08s/it]

Enthalpy components at T = 298.00 K:
E_pot             -16613.371 eV
E_ZPE                  5.697 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.039 eV
Cv_vib (0->T)          0.223 eV
(C_v -> C_p)           0.026 eV
-------------------------------
H                 -16607.348 eV
Entropy components at T = 298.00 K and P = 101325.0 Pa:
                           S               T*S
S_trans (1 bar)    0.0017995 eV/K        0.536 eV
S_rot              0.0013489 eV/K        0.402 eV
S_elec             0.0000000 eV/K        0.000 eV
S_vib              0.0014284 eV/K        0.426 eV
S (1 bar -> P)    -0.0000011 eV/K       -0.000 eV
-------------------------------------------------
S                  0.0045757 eV/K        1.364 eV
Enthalpy components at T = 298.00 K:
E_pot             -16613.371 eV
E_ZPE                  5.697 eV
Cv_trans (0->T)        0.039 eV
Cv_rot (0->T)          0.039 eV
Cv_vib (0->T)          0.223 eV
(C_v -> C_p)           0.026 eV
-------------------------

## Other usages

For other usages, such as running from the command line, SPE calculation, geometry optimization and tautomer searching, please check the [documentation](https://auto3d.readthedocs.io/en/latest/index.html).